In [1]:
import numpy as np

In [2]:
import pyomo.environ as pyo

In [3]:
model = pyo.ConcreteModel()


# technologies
#   - Wind
#       - class II
#       - class III
#   - Solar
#   - Electrolyzer
#   - Methanol Synthesis
#   - Direct Air Capture
# 
# storages
#   - Electricity (battery)
#   - H2
#   - CO2


# What is the goal?
#   - electricity?
#   - Methanol
#   - Ethanol
#   - H2


num_timestamps = 8760  # one year, hourly
num_technologies = 2

model.timestamps = pyo.RangeSet(0, num_timestamps - 1) # note: first timestamp is missing here!

technologies = ['wind_I' ,'wind_II', 'solar', 'storage_electricity']
num_technologies = len(technologies)

model.capacities = pyo.Var(technologies, domain=pyo.NonNegativeReals)
unit_costs = np.random.rand(num_technologies)

In [4]:
# capacity factor for each technology and each hour in a year
profiles = np.random.rand(num_technologies, num_timestamps)

In [5]:
objective = sum(unit_cost * model.capacities[technology]
                for unit_cost, technology in  zip(unit_costs, technologies))

In [6]:
min_generation = 3

In [7]:
# storage is the storage level

In [8]:
# this is the difference to level of storage to time stamp -1, i.e. 
model.storage_electricity = pyo.Var(model.timestamps, domain=pyo.Reals)

In [9]:
storage_electricity_max_in = 2
storage_electricity_max_out = 2

In [10]:
def storage_electricity_max_in_rule(model, t):
    return  (model.storage_electricity[t] - model.storage_electricity[t-1]
                      <= storage_electricity_max_in * model.capacities['storage_electricity'])

In [12]:
model.storage_electricity_max_in = pyo.Constraint(pyo.RangeSet(1, num_timestamps - 1),
                                                  rule=storage_electricity_max_in_rule)

In [15]:
# XXX is it necessary to assume a certain storage level for t=0?

In [16]:
def storage_electricity_max_out_rule(model, t):
    return  (model.storage_electricity[t-1] - model.storage_electricity[t]
                      <= storage_electricity_max_out * model.capacities['storage_electricity'])

In [17]:
model.storage_electricity_max_out = pyo.Constraint(pyo.RangeSet(1, num_timestamps - 1),
                                                  rule=storage_electricity_max_out_rule)

In [20]:
model.generation = pyo.Constraint(expr=min_generation <= sum(model.capacities[technology]
                                                             for technology in technologies))

In [21]:
# storage input/output

In [22]:
model.OBJ = pyo.Objective(expr=objective)

In [23]:
opt = pyo.SolverFactory('glpk')

In [24]:
opt.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.717453680398835, 'Upper bound': 0.717453680398835, 'Number of objectives': 1, 'Number of constraints': 17520, 'Number of variables': 8765, 'Number of nonzeros': 52559, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.12181282043457031}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [25]:
pwd

'/home/p/reFUEL/costs-vs-fuels'

In [13]:
model.write(filename = "asdf.mps", io_options = {"symbolic_solver_labels":True})

('asdf.mps', 139748445235568)

In [26]:
model.solutions.solutions

In [11]:
model.display()

Model unknown

  Variables:
    capacities : Size=2, Index=capacities_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :  None : False : False : NonNegativeReals
          1 :     0 :   3.0 :  None : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 0.7239069800451068

  Constraints:
    generation : Size=1
        Key  : Lower : Body : Upper
        None :   3.0 :  3.0 :  None
